In [24]:
import pandas as pd
import numpy as np
from acquire import get_zillow_data
import wrangle_zillow

In [2]:
def fill_nulls(df):
    
    df.poolcnt = df.poolcnt.fillna(0)
    df.fireplacecnt = df.fireplacecnt.fillna(0)
    df.heatingorsystemdesc = df.heatingorsystemdesc.fillna('None')
    df.unitcnt = df.unitcnt.fillna(1)
    return df

In [3]:
def remove_outliers(df):
    #filter out bedrooms and bathrooms == 0
    df = df[(df.bedroomcnt > 0) & (df.bedroomcnt <= 7) & (df.bathroomcnt > 0) & (df.bathroomcnt <= 7)]
    #filter out houses less than 400 square feet
    df = df[(df.calculatedfinishedsquarefeet > 400) & (df.calculatedfinishedsquarefeet < 7000)]
    #filter out all units not equal to 1
    df = df[df.unitcnt == 1]
    #removing heating or system source outliers
    df = df[~df.heatingorsystemdesc.isin(['Yes', 'Gravity', 'Radiant', 'Baseboard', 'Solar', 'Forced air'])]
    return df

In [4]:
def create_features(df):
    df['age'] = 2017 - df.yearbuilt
    # create taxrate variable
    df['taxrate'] = df.taxamount/df.taxvaluedollarcnt
    # create acres variable
    df['acres'] = df.lotsizesquarefeet/43560
    # dollar per square foot-structure
    df['structure_dollar_per_sqft'] = df.structuretaxvaluedollarcnt/df.calculatedfinishedsquarefeet
    # dollar per square foot-land
    df['land_dollar_per_sqft'] = df.landtaxvaluedollarcnt/df.lotsizesquarefeet
    # ratio of beds to baths
    df['bed_bath_ratio'] = df.bedroomcnt/df.bathroomcnt
    #changing numbered labels into appropriate names
    df['county'] = df.fips.replace([6037, 6059, 6111],['los_angeles', 'orange', 'ventura'])
    df.heatingorsystemdesc = df.heatingorsystemdesc.replace(['Central', 'Floor/Wall', 'None'], ['central_heating', 'floor_wall_heating', 'no_heating'])
    #creating dummy variables
    county_df = pd.get_dummies(df.county)
    heating_or_system_df = pd.get_dummies(df.heatingorsystemdesc)
    #adding dummies back into main dataframe
    df = pd.concat([df, county_df, heating_or_system_df], axis=1)
    #duplicating logerror so it will be at the end of the list
    df['error'] = df.logerror
    #filter out outliers on new features
    df = df[(df.acres < 10) & (df.taxrate < .05)]
    #drop duplicate columns
    df = df.drop(columns = ['bathroomcnt', 'county', 'taxamount', 'taxvaluedollarcnt', 
                       'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 
                       'yearbuilt', 'lotsizesquarefeet', 'logerror', 'heatingorsystemdesc'])
    return df

In [5]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    '''
    This function takes in a Dataframe, 
    proportion(0-1) of nulls required for a column
    and a proportion(0-1) of nulls required for rows
    then returns a dataframe without the nulls 
    under the threshold
    '''
    #setting threshold for row, only accepts integer
    thresh_row = int(round(prop_required_column*df.shape[0],0))
    #dropping nulls under threshold
    df.dropna(axis=1, thresh=thresh_row, inplace=True)
    #setting threshold for columns, only accepts integer
    thresh_col = int(round(prop_required_row*df.shape[1],0))
    #dropping nulls under threshold
    df.dropna(axis=0, thresh=thresh_col, inplace=True)
    return df 

In [6]:
df = get_zillow_data()


In [7]:
#filling nulls with appropriate values
df = fill_nulls(df)

In [8]:
df = remove_outliers(df)


In [9]:
df = create_features(df)


In [10]:
#filter out columns and rows with more than 40% null values
df = handle_missing_values(df, .6, .6)

In [ ]:
#drop duplicate or unnecessary columns
df = df.drop(columns = ['propertylandusetypeid', 'propertycountylandusecode', 'propertylandusedesc',
                             'calculatedbathnbr', 'finishedsquarefeet12', 'heatingorsystemtypeid', 
                            'id', 'fips', 'fullbathcnt', 'propertyzoningdesc', 'unitcnt',
                            'regionidcounty', 'id.1', 'assessmentyear', 'censustractandblock', 'rawcensustractandblock', 'buildingqualitytypeid'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50939 entries, 0 to 52441
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      50939 non-null  int64  
 1   propertylandusetypeid         50939 non-null  float64
 2   heatingorsystemtypeid         32714 non-null  float64
 3   id                            50939 non-null  int64  
 4   bedroomcnt                    50939 non-null  float64
 5   buildingqualitytypeid         33084 non-null  float64
 6   calculatedbathnbr             50924 non-null  float64
 7   calculatedfinishedsquarefeet  50939 non-null  float64
 8   finishedsquarefeet12          50785 non-null  float64
 9   fips                          50939 non-null  float64
 10  fireplacecnt                  50939 non-null  float64
 11  fullbathcnt                   50924 non-null  float64
 12  latitude                      50939 non-null  float64
 13  l

In [12]:
df.dropna(inplace = True)


In [13]:
def zillow_split(df):
    '''
    This function splits a dataframe into train, validate, and test sets
    '''
    train_and_validate, test = train_test_split(df, train_size=.8, random_state=123)
    train, validate = train_test_split(train_and_validate, train_size = .7, random_state=123)
    return train, validate, test

In [14]:
from sklearn.model_selection import train_test_split


In [27]:
train, validate, test = zillow_split(df)


In [16]:
cols_fixed = ['buildingqualitytypeid', 'regionidcity', 'regionidzip', 'age']
for col in cols_fixed:
    mode = int(train[col].mode())
    train[col].fillna(value = mode, inplace = True)
    validate[col].fillna(value = mode, inplace = True)
    test[col].fillna(value = mode, inplace = True)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [17]:
#missing continuous values will be replaced with the median
cols_cont = ['taxrate', 'acres', 'structure_dollar_per_sqft', 'land_dollar_per_sqft']
for col in cols_cont:
    median = train[col].median()
    train[col].fillna(median, inplace=True)
    validate[col].fillna(median, inplace=True)
    test[col].fillna(median, inplace=True)

In [18]:
train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17859 entries, 1932 to 7053
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      17859 non-null  int64  
 1   propertylandusetypeid         17859 non-null  float64
 2   heatingorsystemtypeid         17859 non-null  float64
 3   id                            17859 non-null  int64  
 4   bedroomcnt                    17859 non-null  float64
 5   buildingqualitytypeid         17859 non-null  float64
 6   calculatedbathnbr             17859 non-null  float64
 7   calculatedfinishedsquarefeet  17859 non-null  float64
 8   finishedsquarefeet12          17859 non-null  float64
 9   fips                          17859 non-null  float64
 10  fireplacecnt                  17859 non-null  float64
 11  fullbathcnt                   17859 non-null  float64
 12  latitude                      17859 non-null  float64
 13 

In [19]:
validate.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7655 entries, 46412 to 3447
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      7655 non-null   int64  
 1   propertylandusetypeid         7655 non-null   float64
 2   heatingorsystemtypeid         7655 non-null   float64
 3   id                            7655 non-null   int64  
 4   bedroomcnt                    7655 non-null   float64
 5   buildingqualitytypeid         7655 non-null   float64
 6   calculatedbathnbr             7655 non-null   float64
 7   calculatedfinishedsquarefeet  7655 non-null   float64
 8   finishedsquarefeet12          7655 non-null   float64
 9   fips                          7655 non-null   float64
 10  fireplacecnt                  7655 non-null   float64
 11  fullbathcnt                   7655 non-null   float64
 12  latitude                      7655 non-null   float64
 13 

In [20]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6379 entries, 26893 to 12755
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      6379 non-null   int64  
 1   propertylandusetypeid         6379 non-null   float64
 2   heatingorsystemtypeid         6379 non-null   float64
 3   id                            6379 non-null   int64  
 4   bedroomcnt                    6379 non-null   float64
 5   buildingqualitytypeid         6379 non-null   float64
 6   calculatedbathnbr             6379 non-null   float64
 7   calculatedfinishedsquarefeet  6379 non-null   float64
 8   finishedsquarefeet12          6379 non-null   float64
 9   fips                          6379 non-null   float64
 10  fireplacecnt                  6379 non-null   float64
 11  fullbathcnt                   6379 non-null   float64
 12  latitude                      6379 non-null   float64
 13

In [21]:
def scaled_zillow_columns(cached = True):
    '''
    This function uses a MinMaxScaler to scale numeric columns
    from the wrangle_zillow function
    '''
    train, validate, test = wrangle_zillow()
    columns_to_scale= ['bedroomcnt', 'buildingqualitytypeid', 'calculatedfinishedsquarefeet', 'fireplacecnt', 'latitude', 'longitude', 'poolcnt', 'regionidcity', 'regionidzip', 'roomcnt', 'age', 'taxrate', 'taxrate', 'acres', 'structure_dollar_per_sqft', 'land_dollar_per_sqft', 'bed_bath_ratio']
    #initialize scaler function
    scaler = sklearn.preprocessing.MinMaxScaler()
    #adds '_scaled' to columns that will be scaled
    new_column_names = [c + '_scaled' for c in columns_to_scale]
    #fitting columns to be scaled
    scaler.fit(train[columns_to_scale])
    #adding scaled columns back into their respective dataframes
    train = pd.concat([
        train,
        pd.DataFrame(scaler.transform(train[columns_to_scale]), columns=new_column_names, index=train.index),
    ], axis=1)
    validate = pd.concat([
        validate,
        pd.DataFrame(scaler.transform(validate[columns_to_scale]), columns=new_column_names, index=validate.index),
    ], axis=1)
    test = pd.concat([
        test,
        pd.DataFrame(scaler.transform(test[columns_to_scale]), columns=new_column_names, index=test.index),
    ], axis=1)
    
    train = train.drop(columns = columns_to_scale)
    validate= validate.drop(columns = columns_to_scale)
    test = test.drop(columns = columns_to_scale)

In [28]:
train_scaled, validate_scaled, test_scaled = wrangle_zillow.scaled_zillow_columns()


In [29]:
train_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27955 entries, 32407 to 16021
Data columns (total 26 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   parcelid                             27955 non-null  int64  
 1   transactiondate                      27955 non-null  object 
 2   heatingorsystemdesc                  27955 non-null  object 
 3   county                               27955 non-null  object 
 4   los_angeles                          27955 non-null  uint8  
 5   orange                               27955 non-null  uint8  
 6   ventura                              27955 non-null  uint8  
 7   central_heating                      27955 non-null  uint8  
 8   floor_wall_heating                   27955 non-null  uint8  
 9   no_heating                           27955 non-null  uint8  
 10  error                                27955 non-null  float64
 11  bedroomcnt_scaled       